## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_log_error

## Prepare data for model training

In [2]:
with open("../input/workation-price-prediction-preprocess-data-v4/Workation_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

train_df = train_df[~((train_df['Per Person Price'] < 7.3) | (np.exp(train_df['Per Person Price']) > 150000))].copy()

del processed_data
gc.collect()

31

In [3]:
Xtrain = train_df.loc[:, train_df.columns != 'Per Person Price'].copy()
Ytrain = train_df['Per Person Price'].copy()
Ytrain_strat = pd.qcut(train_df['Per Person Price'].values, q=10, labels=range(0,10))
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (20989, 5752) 
Ytrain: (20989,) 
Xtest: (9000, 5752)


0

## Build and validate the model

In [4]:
FOLD = 10
NUM_SEED = 3

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], NUM_SEED))
y_pred_final_cb = np.zeros((Xtest.shape[0], NUM_SEED))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain_strat)):
        counter += 1

        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]

        model = CatBoostRegressor(
            objective='RMSE',
            eval_metric='RMSE',
            num_boost_round=8000,
            learning_rate=0.014,
            reg_lambda=0.0487,
            bootstrap_type='Poisson',
            subsample=0.922,
            max_depth=7, 
            grow_policy='Lossguide',
            min_data_in_leaf=6, 
            max_leaves=957,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  early_stopping_rounds=200, verbose=100)

        y_pred = model.predict(val_x)
        y_pred_meta_cb[val, sidx] += y_pred
        y_pred_final_cb[:, sidx] += model.predict(Xtest)
        
        score = np.sqrt(mean_squared_log_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final_cb = y_pred_final_cb / float(FOLD)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

0:	learn: 0.5345557	test: 0.5298913	best: 0.5298913 (0)	total: 207ms	remaining: 27m 33s
100:	learn: 0.2273945	test: 0.2371229	best: 0.2371229 (100)	total: 9.55s	remaining: 12m 27s
200:	learn: 0.1608326	test: 0.1856100	best: 0.1856100 (200)	total: 17.6s	remaining: 11m 24s
300:	learn: 0.1407848	test: 0.1748278	best: 0.1748278 (300)	total: 24.8s	remaining: 10m 33s
400:	learn: 0.1315461	test: 0.1711070	best: 0.1711070 (400)	total: 30.9s	remaining: 9m 44s
500:	learn: 0.1261183	test: 0.1692967	best: 0.1692967 (500)	total: 36.6s	remaining: 9m 7s
600:	learn: 0.1221064	test: 0.1680102	best: 0.1680102 (600)	total: 41.6s	remaining: 8m 32s
700:	learn: 0.1184122	test: 0.1669234	best: 0.1669234 (700)	total: 46.4s	remaining: 8m 2s
800:	learn: 0.1152452	test: 0.1661583	best: 0.1661583 (800)	total: 51.1s	remaining: 7m 39s
900:	learn: 0.1124489	test: 0.1654654	best: 0.1654626 (899)	total: 56s	remaining: 7m 21s
1000:	learn: 0.1098143	test: 0.1649686	best: 0.1649686 (1000)	total: 1m	remaining: 7m 4s
1100:

In [5]:
np.savez_compressed('./CB_Meta_Features.npz',
                    y_pred_meta_cb=y_pred_meta_cb, 
                    oof_score=oof_score,
                    y_pred_final_cb=y_pred_final_cb)

In [6]:
df = pd.DataFrame(y_pred_final_cb)
df.head()

,0,1,2
0,9.840287,9.841775,9.836069
1,9.329983,9.338839,9.330478
2,8.656160,8.652101,8.651663
3,8.874824,8.871322,8.870665
4,9.965540,9.965354,9.966350
